In [2]:
import requests
import json
import zlib
import base64
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry
from urllib3.exceptions import MaxRetryError

In [3]:
# Compress data before sending it over the network
def compress_data(data):
    """
    Compresses JSON data using zlib and encodes it in base64 for safe network transmission.
    Args:
    data (dict): Data to be compressed and sent.
    Returns:
    str: Compressed data encoded in base64.
    """
    json_data = json.dumps(data).encode('utf-8')
    compressed_data = zlib.compress(json_data)
    return base64.b64encode(compressed_data).decode('utf-8')

In [4]:
# Decompress data received over the network
def decompress_data(compressed_data):
    """
    Decompresses data received in base64 encoded zlib format back to JSON.
    Args:
    compressed_data (bytes): Compressed data encoded in base64 to be decompressed.
    Returns:
    dict: Decompressed data as a Python dictionary.
    """
    data_bytes = base64.b64decode(compressed_data)
    decompressed_data = zlib.decompress(data_bytes)
    return json.loads(decompressed_data.decode('utf-8'))

In [5]:
# Setup a session with retry logic
def setup_session():
    """
    Sets up a requests session with retry logic to handle transient errors in network communication.
    Returns:
    requests.Session: A configured session with retry logic.
    """
    session = requests.Session()
    retries = Retry(total=5, backoff_factor=1, status_forcelist=[502, 503, 504], raise_on_status=False)
    session.mount('https://', HTTPAdapter(max_retries=retries))
    return session

In [6]:
# Example data representing an API payload
example_data = {
    'name': 'Jane Smith',
    'email': 'jane.smith@example.com',
    'phone': '07123 456789'
}

In [10]:
compressed_payload = compress_data(example_data)
compressed_payload

'eJyrVspLzE1VslJQ8krMS1UIzs0syVDSUVBKzU3MzAEJZwGF9YpBwg6pFYm5BTmpesn5uSAlBRn5eWCdBuaGRsYKJqZm5haWSrUAKxQYiw=='

In [8]:
# Simulate sending compressed data to an API
api_url = 'https://httpbin.org/post'
headers = {'Content-Type': 'application/json', 'Content-Encoding': 'gzip'}

In [9]:
# Use the session with retry logic
session = setup_session()
try:
    response = session.post(api_url, data=compressed_payload, headers=headers)
    # Check if response is received and process it
    if response.status_code == 200:
        print("Response received successfully.")
        print("Response JSON:", response.json())
        # Decompress the received data if it is compressed
        received_compressed_data = response.json()['data']
        decompressed_data = decompress_data(received_compressed_data)
        print("Decompressed Data:", decompressed_data)
    else:
        print(f"Failed to fetch data: HTTP {response.status_code}")
except MaxRetryError as e:
    print(f"Max retries exceeded: {e.reason}")

# Cleanup session
session.close()

Response received successfully.
Response JSON: {'args': {}, 'data': 'eJyrVspLzE1VslJQ8krMS1UIzs0syVDSUVBKzU3MzAEJZwGF9YpBwg6pFYm5BTmpesn5uSAlBRn5eWCdBuaGRsYKJqZm5haWSrUAKxQYiw==', 'files': {}, 'form': {}, 'headers': {'Accept': '*/*', 'Accept-Encoding': 'gzip, deflate, br, zstd', 'Content-Encoding': 'gzip', 'Content-Length': '108', 'Content-Type': 'application/json', 'Host': 'httpbin.org', 'User-Agent': 'python-requests/2.32.3', 'X-Amzn-Trace-Id': 'Root=1-681ccafd-795cb0b12d36b55c3d1bffcd'}, 'json': None, 'origin': '90.217.121.18', 'url': 'https://httpbin.org/post'}
Decompressed Data: {'name': 'Jane Smith', 'email': 'jane.smith@example.com', 'phone': '07123 456789'}
